# Grundlagen

In [ ]:
import gym
import gnwrapper #offline auskommentieren
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

def UmgebungErstellen():
    umgebung = gnwrapper.Animation(gym.make('SpaceInvaders-v0')) #online
    #umgebung = gym.make('SpaceInvaders-v0') #offline
    höhe, breite, kanäle = umgebung.observation_space.shape 
    aktionen = umgebung.action_space.n 
    return umgebung, höhe, breite, kanäle, aktionen;

def AktionenAnzeigen(umgebung):
    umgebung.unwrapped.get_action_meanings();
    
def Probedurchlauf(anzahl, umgebung):
    ausgabe= ''
    for anzahl in range(1, anzahl+1):
        state = umgebung.reset()
        done = False
        score = 0 
        schritte = 0
        
        while not done:
            umgebung.render()
            action = random.choice([0,1,2,3,4,5])
            n_state, reward, done, info = umgebung.step(action)
            score+=reward
            schritte+=1
        ausgabe +='Durchlauf:{} Punkte:{} Schritte:{}'.format(anzahl, score, schritte)+'\n'
    umgebung.close()
    print(ausgabe)
    
def TrainingsplanErstellen(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    print("Trainingsplan:")
    #Trainable params gibt die Anzahl der möglichen Kombinationen an  
    model.summary()
    return model

def KIErstellen(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000)
    print("KI erstellt")
    return dqn

def KITrainieren(ki, durchläufe):
    ki.compile(Adam(lr=1e-4))
    ki.fit(umgebung, nb_steps=durchläufe, visualize=False, verbose=2)
    print("KI trainiert")
    return ki

def ErgebnisAusgeben(ergebnis):
    print(np.mean(ergebnis.history['episode_reward']))
    
def KILaden():
    #zum laden muss der Kernel neu gestartet werden und dieser Codeblock nochmals ausgeführt werden, ansonsten gibts Speicherprobleme bei Binder 
    umgebung, höhe, breite, kanäle, aktionen = UmgebungErstellen();
    trainingsplan = TrainingsplanErstellen(höhe, breite, kanäle, aktionen)
    ki = KIErstellen(trainingsplan, aktionen)
    ki.compile(Adam(lr=1e-4))
    ki.load_weights('savedweights/ki_weights.h5f')
    ergebnis = ki.test(umgebung, nb_episodes=10, visualize=True)
    ErgebnisAusgeben(ergebnis)
    print("fertig")
print("geladen")

# Kompletter Durchlauf

In [ ]:
umgebung, höhe, breite, kanäle, aktionen = UmgebungErstellen();

AktionenAnzeigen(umgebung);

Probedurchlauf(3, umgebung);
print("Probedurchlauf fertig")

trainingsplan = TrainingsplanErstellen(höhe, breite, kanäle, aktionen)

ki = KIErstellen(trainingsplan, aktionen)

ki = KITrainieren(ki,1000)

ergebnis = ki.test(umgebung, nb_episodes=10, visualize=True)

ErgebnisAusgeben(ergebnis)





# Trainierter Durchlauf

In [ ]:
KILaden()